In [1]:
%%init_spark
launcher.conf.set("spark.jars.packages", "org.apache.hudi:hudi-spark3.1-bundle_2.12:0.12.2,org.apache.spark:spark-avro_2.12:3.1.3")
launcher.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
launcher.conf.set("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
launcher.conf.set("spark.sql.hive.convertMetastoreParquet", "false")


In [2]:
import org.apache.hudi.QuickstartUtils._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.functions._ 
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.common.model.HoodieRecord
import org.apache.spark.sql.types.{TimestampType, StringType}

// Imports for Reproducing Timestamp to Bigint default conversion
import org.apache.hudi.keygen.ComplexKeyGenerator
import org.apache.hudi.common.model.DefaultHoodieRecordPayload

Intitializing Scala interpreter ...

Spark Web UI available at http://cluster-4a27-m:8088/proxy/application_1674530151748_0006
SparkContext available as 'sc' (version = 3.1.3, master = yarn, app id = application_1674530151748_0006)
SparkSession available as 'spark'


import org.apache.hudi.QuickstartUtils._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.common.model.HoodieRecord
import org.apache.spark.sql.types.{TimestampType, StringType}
import org.apache.hudi.keygen.ComplexKeyGenerator
import org.apache.hudi.common.model.DefaultHoodieRecordPayload


In [3]:
spark.version

res0: String = 3.1.3


In [4]:
val full_load = "gs://data-lake-cdc/data/demo_hudi_delta_test/2023/01/12/04/04/8c0972982087b235becf90b60a52554e7f3774ff_mysql-backfill-fulldump_-1046201224_6_0.avro"
val cdc_load = "gs://data-lake-cdc/data/demo_hudi_delta_test/2023/01/12/04/10/8c0972982087b235becf90b60a52554e7f3774ff_mysql-cdc-binlog_-1046201224_3_0.avro"

full_load: String = gs://data-lake-cdc/data/demo_hudi_delta_test/2023/01/12/04/04/8c0972982087b235becf90b60a52554e7f3774ff_mysql-backfill-fulldump_-1046201224_6_0.avro
cdc_load: String = gs://data-lake-cdc/data/demo_hudi_delta_test/2023/01/12/04/10/8c0972982087b235becf90b60a52554e7f3774ff_mysql-cdc-binlog_-1046201224_3_0.avro


### Understanding Datastream CDC

In [5]:
val df = spark.read.format("avro").load(full_load)
df.show(numRows=df.count().toInt, truncate = 0, vertical = true)

-RECORD 0----------------------------------------------------------------------------
 uuid             | 4f3563ca-0213-4df7-9f66-428200000000                             
 read_timestamp   | 2023-01-12 04:04:45                                              
 source_timestamp | 2023-01-12 04:04:45                                              
 object           | demo_hudi_delta_test                                             
 read_method      | mysql-backfill-fulldump                                          
 stream_name      | projects/293162740476/locations/us-central1/streams/mysql-stream 
 schema_key       | 8c0972982087b235becf90b60a52554e7f3774ff                         
 sort_keys        | [{null, 1673496285000}, {, null}, {null, 0}]                     
 source_metadata  | {hudi_delta_test, demo, [pk_id], , 0, INSERT, false}             
 payload          | {1, apple, 10, 2023-01-12 04:01:18, 2023-01-12 04:01:18}         
-RECORD 1---------------------------------------------

df: org.apache.spark.sql.DataFrame = [uuid: string, read_timestamp: timestamp ... 8 more fields]


In [ ]:
val df_cdc = spark.read.format("avro").load(cdc_load)
df_cdc.show(numRows=df_cdc.count().toInt, truncate = 0, vertical = true)

## Using Apache HUDI

### Using Copy on Write

In [ ]:
val hudiTableName = "hudi_cow"
val hudiTablePath = "gs://data-lake-cdc/data/hudi_cow"

In [ ]:
val fullDF = spark.read.format("avro").load(full_load)
fullDF.select("payload").show(numRows=fullDF.count().toInt, truncate = 0)

In [ ]:
val explodedFullDF = fullDF.select("payload.*", "*")
explodedFullDF.show(numRows=explodedFullDF.count().toInt, truncate = 0, vertical = true)

In [ ]:
explodedFullDF.printSchema

### Converting to timestamp

In [ ]:
val explodedTsFullDF = explodedFullDF
                            .withColumn("updated_at", unix_timestamp(col("updated_at"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("created_at", unix_timestamp(col("created_at"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("read_timestamp", unix_timestamp(col("read_timestamp"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("source_timestamp", unix_timestamp(col("source_timestamp"), "MM-dd-yyyy HH:mm:ss"))
                            // .drop("updated_at")
                            // .drop("created_at")
                            // .drop("read_timestamp")
                            // .drop("source_timestamp")




In [ ]:
explodedTsFullDF.show(numRows=explodedTsFullDF.count().toInt, truncate = 0, vertical = true)

### Saving hudi_cow table

In [ ]:
spark.sql("show tables").show(truncate=false)

In [ ]:
// Clean previews savings
spark.sql("DROP TABLE IF EXISTS "+hudiTableName)

In [ ]:
explodedTsFullDF.write.format("hudi")
    .option(RECORDKEY_FIELD_OPT_KEY, "pk_id")
    .option(PARTITIONPATH_FIELD_OPT_KEY, "created_at")
    .option(TABLE_NAME, hudiTableName)
    .option(OPERATION_OPT_KEY, INSERT_OPERATION_OPT_VAL)
    .option(STORAGE_TYPE_OPT_KEY, "COPY_ON_WRITE")
    .option(PRECOMBINE_FIELD_OPT_KEY, "updated_at")
    .option(HIVE_SYNC_ENABLED_OPT_KEY, "true")
    .option(HIVE_TABLE_OPT_KEY, hudiTableName)
    .option(HIVE_PARTITION_FIELDS_OPT_KEY, "created_at")
    .option(HIVE_ASSUME_DATE_PARTITION_OPT_KEY, "false")
    .option("hoodie.datasource.hive_sync.support_timestamp", "true")
    .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY, classOf[MultiPartKeysValueExtractor].getName)
    .mode(Overwrite)
    .save(hudiTablePath)

In [ ]:
spark.sql("select * from hudi_cow").show(numRows=10, truncate=0, vertical=true)

In [ ]:
df_cdc.show(numRows=10, truncate=0, vertical=true)

In [ ]:
val cdcTsDF = df_cdc
                            .select("payload.*", "*")
                            .withColumn("updated_at", unix_timestamp(col("updated_at"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("created_at", unix_timestamp(col("created_at"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("read_timestamp", unix_timestamp(col("read_timestamp"), "MM-dd-yyyy HH:mm:ss"))
                            .withColumn("source_timestamp", unix_timestamp(col("source_timestamp"), "MM-dd-yyyy HH:mm:ss"))

In [ ]:
cdcTsDF.show(numRows=10, truncate=0, vertical=true)

### Updating hudi_cow table

In [ ]:
cdcTsDF.write.format("hudi")
    .option(RECORDKEY_FIELD_OPT_KEY, "pk_id")
    .option(PARTITIONPATH_FIELD_OPT_KEY, "created_at")
    .option(TABLE_NAME, hudiTableName) 
    .option(STORAGE_TYPE_OPT_KEY, "COPY_ON_WRITE")
    .option(PRECOMBINE_FIELD_OPT_KEY, "updated_at")
    .option(HIVE_SYNC_ENABLED_OPT_KEY, "true")
    .option(HIVE_TABLE_OPT_KEY, hudiTableName)
    .option(HIVE_PARTITION_FIELDS_OPT_KEY, "created_at")
    .option(HIVE_ASSUME_DATE_PARTITION_OPT_KEY, "false")
    .option("hoodie.datasource.hive_sync.support_timestamp", "true")
    .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY, classOf[MultiPartKeysValueExtractor].getName)
    .option(OPERATION_OPT_KEY,UPSERT_OPERATION_OPT_VAL)
    .mode(Append).save(hudiTablePath)

In [ ]:
spark.sql("select * from hudi_cow").show(numRows=10, truncate=0, vertical=true)

In [ ]:
val fullDFUpdated = spark.read.format("hudi").load(hudiTablePath)
fullDFUpdated.show(numRows=10, truncate=0, vertical=true)

### Saving hudi_mor table

In [ ]:
val hudiTableNameMor = "hudi_mor"
val hudiTablePathMor = "gs://data-lake-cdc/data/hudi_mor"

In [ ]:
spark.sql("show tables").show(truncate=false)

In [ ]:
// Clean previews savings
spark.sql("DROP TABLE IF EXISTS "+hudiTableNameMor+"_ro")
spark.sql("DROP TABLE IF EXISTS "+hudiTableNameMor+"_rt")

In [ ]:
val hudiOptions = Map[String,String](
  RECORDKEY_FIELD_OPT_KEY -> "pk_id",
  PARTITIONPATH_FIELD_OPT_KEY -> "created_at",
  TABLE_NAME -> hudiTableNameMor,
  OPERATION_OPT_KEY -> INSERT_OPERATION_OPT_VAL,
  STORAGE_TYPE_OPT_KEY -> "MERGE_ON_READ",
  PRECOMBINE_FIELD_OPT_KEY -> "updated_at",
  HIVE_SYNC_ENABLED_OPT_KEY -> "true",
  HIVE_TABLE_OPT_KEY -> hudiTableNameMor,
  HIVE_PARTITION_FIELDS_OPT_KEY -> "created_at",    
  HIVE_ASSUME_DATE_PARTITION_OPT_KEY -> "false",
  "hoodie.datasource.hive_sync.support_timestamp" -> "true",  
  HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
)

In [ ]:
explodedTsFullDF.write.format("hudi")
    .options(hudiOptions)
    .mode(Overwrite)
    .save(hudiTablePathMor)

In [ ]:
spark.sql("select * from hudi_mor_ro").show(numRows=10, truncate=0, vertical=true)

In [ ]:
spark.sql("select * from hudi_mor_rt").show(numRows=10, truncate=0, vertical=true)

### Updating hudi_mor table

In [ ]:
val hudiOptionsMorUpdate = Map[String,String](
  RECORDKEY_FIELD_OPT_KEY -> "pk_id",
  PARTITIONPATH_FIELD_OPT_KEY -> "created_at",
  TABLE_NAME -> hudiTableNameMor,
  OPERATION_OPT_KEY -> UPSERT_OPERATION_OPT_VAL,
  STORAGE_TYPE_OPT_KEY -> "MERGE_ON_READ",
  PRECOMBINE_FIELD_OPT_KEY -> "updated_at",
  HIVE_SYNC_ENABLED_OPT_KEY -> "true",
  HIVE_TABLE_OPT_KEY -> hudiTableNameMor,
  HIVE_PARTITION_FIELDS_OPT_KEY -> "created_at",    
  HIVE_ASSUME_DATE_PARTITION_OPT_KEY -> "false",
  "hoodie.datasource.hive_sync.support_timestamp" -> "true",  
  HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY -> classOf[MultiPartKeysValueExtractor].getName
)


cdcTsDF.write.format("hudi")
    .options(hudiOptionsMorUpdate)
    .mode(Append).save(hudiTablePathMor)

In [ ]:
spark.sql("select * from hudi_mor_ro").show(numRows=10, truncate=0, vertical=true)

In [ ]:
spark.sql("select * from hudi_mor_rt").show(numRows=10, truncate=0, vertical=true)